In [992]:
import random
from itertools import combinations


# Лабораторная работа №5

## Имитационная модель сети Петри

In [993]:
class PetriNet:
    def __init__(self, transitions, marking):
        self.places = list([Place(m, i) for i, m in enumerate(marking)])
        self.transitions = list([
            Transition(
                [OutArc(self.places[i]) for i in outs],
                [InArc(self.places[i]) for i in ins])
            for outs, ins in transitions
        ])
        # print(f"Places: {self.places}\tTransitions: {self.transitions}")
        self.finished = False
        self.markings = [self.get_marking()]


    def get_marking(self):
        return [x.holding for x in self.places]

    def run(self, info=False):
        i = 0
        while not self.finished and i < 25:
            current_marking = self._run_once(info)
            if not current_marking:
                self.finished = True
                break
            yield current_marking
            i += 1

    def _run_once(self, info):
        fired_transitions = set()
        while len(fired_transitions) < len(self.transitions):
            t_num = random.randint(0, len(self.transitions) - 1)
            t = self.transitions[t_num]
            if t.fire():
                if info:
                    # display_table([t_num, t, self.get_marking()])
                    display_table([[str(t_num), str(t), str(self.get_marking())]])
                    # print(
                    #     f"Firing transition {t_num}: {t} \tMarking {self.get_marking()}")
                    # print(f"Current marking {self.get_marking()}")
                return self.get_marking()
            else:
                fired_transitions.add(t_num)
        return None


In [994]:
class Transition:
    def __init__(self, out_arcs, in_arcs):
        self.out_arcs = set(out_arcs)
        self.in_arcs = set(in_arcs)
        self.arcs = self.out_arcs.union(in_arcs)
        # print(f"Out arcs: {self.out_arcs}\nIn arcs: {self.in_arcs}")
        # print(f"Arcs: {self.arcs}")

    def fire(self):
        if self.not_blocked():
            for arc in self.arcs:
                arc.trigger()
            return True
        return False

    def not_blocked(self):
        return all(arc.non_blocking() for arc in self.out_arcs)

    def get_out_arc(self, place):
        return [a for a in self.out_arcs if a.place == place][0]

    def get_in_arc(self, place):
        return [a for a in self.in_arcs if a.place == place][0]

    def __str__(self):
        return f"{list(self.out_arcs)} -> {list(self.in_arcs)}"

    __repr__ = __str__


In [995]:
class Place:
    def __init__(self, holding, id=None):
        self.holding = holding
        self.id = id

    def __str__(self):
        return f"#{self.id+1}({self.holding})"

    __repr__ = __str__


In [996]:
class Arc:
    def __init__(self, place: Place, amount=1):
        self.place = place
        self.amount = amount

    def __str__(self):
        return f"{self.place}" + (f"[{self.amount}]" if self.amount > 1 else "")

    __repr__ = __str__


In [997]:
class OutArc(Arc):
    def trigger(self):
        self.place.holding -= self.amount

    def non_blocking(self):
        return self.place.holding >= self.amount


In [998]:
class InArc(Arc):
    def trigger(self):
        self.place.holding += self.amount


![](./1.png)

In [999]:
# 1
marking1 = [0, 0, 1, 1, 0]
transitions1 = [
    ([3], [0]),
    ([2], [1]),
    ([0, 1], [2, 3]),
    ([3], [4])
]

![](./2.png)

In [1000]:
# 2
marking2 = [1, 0, 0, 0, 0, 1]
transitions2 = [
    ([0], [1, 2, 5]),
    ([1], [3]),
    ([2], [4]),
    ([3, 4, 5], [0])
]


## Характеристики сети Петри

### Ограниченность

In [1001]:
def bounded_with(net: PetriNet):
    markings = [net.get_marking()]
    markings += [m for m in net.run()]
    return max([max(m) for m in markings])


### Безопасность

In [1002]:
def is_safe(net: PetriNet):
    return bounded_with(net) == 1


### Консервативность

In [1003]:
def is_conservative(net: PetriNet):
    s0 = sum(net.get_marking())
    for marking in net.run():
        s = sum(marking)
        if s != s0:
            return False
    return True


### Живая / не живая

In [1004]:
def is_alive(net: PetriNet):
    for _ in net.run():
        pass
    return not net.finished


### Параллельность

In [1005]:
def check_parralel(net: PetriNet):
    not_blocked_ts = [t for t in net.transitions if t.not_blocked()]
    if len(not_blocked_ts) <= 1:
        return False
    for t1, t2 in combinations(not_blocked_ts, 2):
        places1 = set(place for place in t1.out_arcs)
        places2 = set(place for place in t2.out_arcs)
        if places1 != places2:
            return True
        mutual_places = places1.intersection(places2)
        if all(p.holding >= t1.get_out_arc(p).amount + t2.get_out_arc(p).amount for p in mutual_places):
            return True
    return False


def is_parallel(net: PetriNet):
    if check_parralel(net):
        return True
    for _ in net.run():
        if check_parralel(net):
            return True
    return False


### Свободного выбора

In [1006]:
def is_free_choice(net: PetriNet):
    for t1, t2 in combinations(net.transitions, 2):
        places1 = set(arc.place for arc in t1.out_arcs)
        places2 = set(arc.place for arc in t2.out_arcs)
        if places1.intersection(places2):
            return True
    return False


### Маркированная

In [1007]:
def is_marked(net: PetriNet):
    for place in net.places:
        in_arcs = [a for t in net.transitions
                   for a in t.in_arcs if a.place == place]
        out_arcs = [a for t in net.transitions
                    for a in t.out_arcs if a.place == place]
        if len(out_arcs) != len(in_arcs) != 1:
            return False
    return True


### Табличное представлние


In [1008]:
from IPython.display import HTML, display


def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4></td>" % (field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))


data = [['Опсиание', 'Схема 1', 'Схема 2'],
        ['Ограниченость', bounded_with(PetriNet(transitions1, marking1)), bounded_with(
            PetriNet(transitions2, marking2))],
        ['Безопастность', is_safe(PetriNet(transitions1, marking1)), is_safe(
            PetriNet(transitions2, marking2))],
        ['Консервативность', is_conservative(PetriNet(
            transitions1, marking1)), is_conservative(PetriNet(transitions2, marking2))],
        ['Живая', is_alive(PetriNet(transitions1, marking1)),
         is_alive(PetriNet(transitions2, marking2))],
        ['Параллельность', is_parallel(PetriNet(transitions1, marking1)), is_parallel(
            PetriNet(transitions2, marking2))],
        ['Маркированная', is_free_choice(PetriNet(transitions1, marking1)), is_free_choice(PetriNet(transitions2, marking2))]]

display_table(data)


Опсиание,Схема 1,Схема 2
Ограниченость,1,2
Безопастность,True,False
Консервативность,True,False
Живая,False,True
Параллельность,True,True
Маркированная,True,False


### Полное исследование сети #1

![](https://imgur.com/uVBQESS.png)

In [1009]:
def net1():
    return PetriNet(transitions1, marking1)

display_table([["Запуск","Переход","Обновленная маркировка"]])

for _ in net1().run(info=True):
    pass



Запуск,Переход,Обновленная маркировка


1,[#3(0)] -> [#2(1)],"[0, 1, 0, 1, 0]"


3,[#4(0)] -> [#5(1)],"[0, 1, 0, 0, 1]"


### Полное исследование сети #2

![](https://imgur.com/2311NKm.png)

In [1010]:
def net2():
    return PetriNet(transitions2, marking2)
print()
display_table([["Запуск","Переход","Обновленная маркировка"]])
for _ in net2().run(info=True):
    pass



Запуск,Переход,Обновленная маркировка


0,"[#1(0)] -> [#6(2), #2(1), #3(1)]","[0, 1, 1, 0, 0, 2]"


2,[#3(0)] -> [#5(1)],"[0, 1, 0, 0, 1, 2]"


1,[#2(0)] -> [#4(1)],"[0, 0, 0, 1, 1, 2]"


3,"[#5(0), #4(0), #6(1)] -> [#1(1)]","[1, 0, 0, 0, 0, 1]"


...

0,"[#1(0)] -> [#6(2), #2(1), #3(1)]","[0, 1, 1, 0, 0, 2]"


1,[#2(0)] -> [#4(1)],"[0, 0, 1, 1, 0, 2]"


2,[#3(0)] -> [#5(1)],"[0, 0, 0, 1, 1, 2]"


3,"[#5(0), #4(0), #6(1)] -> [#1(1)]","[1, 0, 0, 0, 0, 1]"


0,"[#1(0)] -> [#6(2), #2(1), #3(1)]","[0, 1, 1, 0, 0, 2]"
